# Deep Learning

### Loading Libraries

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
from matplotlib .pyplot import subplots

# Scikit-Learn
from sklearn. pipeline import Pipeline
from sklearn. model_selection import KFold
from sklearn. preprocessing import StandardScaler
from sklearn. model_selection import (train_test_split, GridSearchCV)
from sklearn. linear_model import (LinearRegression, LogisticRegression, Lasso)

# ISLP
from ISLP import load_data
from ISLP.models import ModelSpec as MS

# JavaScript Object Notation
import json

# Glob
from glob import glob

#### Torch-Specific Imports

In [ ]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

In [ ]:
from torchinfo import summary
from torchvision.io import read_image
from torchmetrics import (MeanAbsoluteError, R2Score)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In [ ]:
# from pytorch_lightning.utilities.seed import seed_everything

from pytorch_lightning import seed_everything

seed_everything(0, workers=True)

torch.use_deterministic_algorithms(True, warn_only =True)

In [ ]:
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50, ResNet50_Weights)
from torchvision.transforms import (Resize, Normalize, CenterCrop,ToTensor)

In [ ]:
from ISLP.torch import (SimpleDataModule, SimpleModule, ErrorTracker, rec_num_workers)

In [ ]:
from ISLP.torch.imdb import (load_lookup, load_tensor, load_sparse, load_sequential)

### Single Layer Network on Hitters Data

In [ ]:
Hitters = load_data('Hitters').dropna()

n = Hitters.shape [0]

In [ ]:
model = MS(Hitters.columns.drop('Salary'), intercept =False)

X = model.fit_transform(Hitters).to_numpy()
Y = Hitters['Salary']. to_numpy ()

In [ ]:
(X_train,
 X_test,
 Y_train,
 Y_test) = train_test_split(X,
                            Y,
                            test_size=1/3,
                            random_state=1)

#### Linear Models

In [ ]:
hit_lm = LinearRegression().fit(X_train, Y_train)
Yhat_test = hit_lm.predict(X_test)
np.abs( Yhat_test - Y_test).mean()

In [ ]:
scaler = StandardScaler(with_mean=True, with_std=True)
lasso = Lasso(warm_start=True , max_iter=30000)

standard_lasso = Pipeline(steps=[('scaler', scaler),
                                  ('lasso', lasso)])

In [ ]:
X_s = scaler.fit_transform(X_train)
n = X_s.shape[0]

lam_max = np.fabs(X_s.T.dot(Y_train - Y_train.mean ())).max() / n
param_grid = {'alpha': np.exp(np.linspace(0, np.log(0.01), 100))
              * lam_max}

In [ ]:
cv = KFold(10,
           shuffle=True,
           random_state=1)

grid = GridSearchCV(lasso,
                     param_grid,
                    cv=cv,
                    scoring='neg_mean_absolute_error')

grid.fit(X_train, Y_train);

In [ ]:
trained_lasso = grid.best_estimator_

Yhat_test = trained_lasso.predict(X_test)

np.fabs(Yhat_test - Y_test).mean()

#### Specifying a Network: `Classes and Inheritance`

In [ ]:
class HittersModel(nn.Module):
    def __init__(self, input_size):
        super(HittersModel, self).__init__()
        self.flatten = nn.Flatten()
        self. sequential = nn.Sequential(
            nn.Linear(input_size, 50),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear (50, 1))
    
    def forward(self, x):
        x = self.flatten(x)
        return torch.flatten(self.sequential(x))

In [ ]:
hit_model = HittersModel(X.shape[1])

In [ ]:
summary(hit_model,
        input_size=X_train.shape,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

In [ ]:
X_train_t = torch.tensor(X_train.astype(np.float32))
Y_train_t = torch.tensor(Y_train.astype(np.float32))

hit_train = TensorDataset(X_train_t , Y_train_t )

In [ ]:
X_test_t = torch.tensor(X_test.astype(np.float32))
Y_test_t = torch.tensor(Y_test.astype(np.float32))

hit_test = TensorDataset(X_test_t, Y_test_t)

In [ ]:
max_num_workers = rec_num_workers()

In [ ]:
hit_dm = SimpleDataModule(hit_train,
                          hit_test,
                          batch_size=32,
                          num_workers =min(4, max_num_workers),
                          validation = hit_test)

In [ ]:
hit_module = SimpleModule.regression(hit_model,
                                     metrics={'mae': MeanAbsoluteError()})

In [ ]:
hit_logger = CSVLogger('logs', name='hitters')

In [ ]:
hit_trainer = Trainer(deterministic=True,
                      max_epochs=50,
                      log_every_n_steps=5,
                      logger=hit_logger,
                      callbacks=[ErrorTracker()])

hit_trainer .fit(hit_module , datamodule =hit_dm)

In [ ]:
hit_trainer.test(hit_module, datamodule=hit_dm)

In [ ]:
hit_results = pd.read_csv(hit_logger.experiment. metrics_file_path)

In [ ]:
def summary_plot(results,
                 ax,
                 col='loss',
                 valid_legend ='Validation',
                 training_legend ='Training',
                 ylabel='Loss',
                 fontsize =20):
    for (column,
         color,
         label) in zip ([f'train_{col}_epoch',
                         f'valid_{col}'],
                        ['black', 'red'],
                        [training_legend, valid_legend]):
        results.plot(x='epoch',
                     y=column,
                     label=label,
                     marker='o',
                     color=color,
                     ax=ax)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(ylabel)
    return ax

In [ ]:
fig, ax = subplots(1, 1, figsize=(6, 6))

ax = summary_plot(hit_results,
                  ax,
                  col='mae',
                  ylabel='MAE',
                  valid_legend='Validation(=Test)')

ax.set_ylim([0, 400])
ax.set_xticks(np.linspace(0, 50, 11).astype(int));
plt.grid(True)
plt.show()

In [ ]:
hit_model.eval()
preds = hit_module(X_test_t)
torch.abs(Y_test_t - preds).mean()

#### Cleanup

In [ ]:
del(Hitters,
    hit_model, 
    hit_dm,
    hit_logger,
    hit_test, hit_train,
    X, Y,
    X_test, X_train,
    Y_test, Y_train,
    X_test_t, Y_test_t,
    hit_trainer, hit_module)

### Multilayer Network on the MNIST Digit Data